In [91]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as colors
import warnings
warnings.filterwarnings('ignore', category=UserWarning)  
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*laundered field name.*")

import warnings





In [92]:
import os

working_directory = "/Users/arturoramirez/Desktop/Proyecto DATATÓN"
os.chdir(working_directory)

print("Updated Working Directory:", os.getcwd())

Updated Working Directory: /Users/arturoramirez/Desktop/Proyecto DATATÓN


In [93]:
isocrona_15_l = gpd.read_file("datos/final_gdf_15.shp")
isocrona_30_l = gpd.read_file("datos/final_gdf_30.shp")
isocrona_45_l = gpd.read_file("datos/final_gdf_45.shp")

colonias = gpd.read_file("datos/colonias_iecm.shp")
sintesis = gpd.read_file("datos/urbanismo_social_sintesis/urbanismo_social_sintesis.shp")
densisdad = gpd.read_file("datos/densidad_poblacion_2010/densidadpoblacion2010.shp")



colonias = colonias.to_crs(epsg=4326)



In [94]:


# Set CRS for isochrones (if they need to be set, otherwise skip)
isocrona_15_l.crs = 'epsg:4326'
isocrona_30_l.crs = 'epsg:4326'
isocrona_45_l.crs = 'epsg:4326'



# Reproject isochrones only if their CRS was incorrectly set
isocrona_15_l = isocrona_15_l.to_crs(epsg=4326)
isocrona_30_l = isocrona_30_l.to_crs(epsg=4326)
isocrona_45_l = isocrona_45_l.to_crs(epsg=4326)



In [97]:
# Initialize a list to store results
results = []

# Iterate through each colonia
for idx, colonia in colonias.iterrows():
    # Find intersecting isochrones
    intersecting_isochrones = isocrona_15_l[isocrona_15_l.intersects(colonia.geometry)]
    
    # If there are no intersecting isochrones, add 0% coverage and continue
    if intersecting_isochrones.empty:
        results.append({
            'id': colonia['NOMUT'],
            'NOMUT': colonia['NOMUT'],
            'isochrone_coverage': 0
        })
        continue
    
    # Calculate the union of all intersecting isochrones using union_all
    unified_isochrones = intersecting_isochrones.geometry.union_all()
    
    # Calculate the intersection of the unified isochrones with the colonia
    intersection = unified_isochrones.intersection(colonia.geometry)
    
    # Calculate the percentage of coverage
    coverage_percentage = (intersection.area / colonia.geometry.area) * 100
    
    # Ensure the coverage doesn't exceed 100%
    coverage_percentage = min(coverage_percentage, 100)
    
    # Append the result
    results.append({
        'id': colonia['NOMUT'],
        'NOMUT': colonia['NOMUT'],
        'isochrone_coverage': coverage_percentage
    })

# Create a DataFrame from the results
result_df = pd.DataFrame(results)



In [98]:
# Initialize a list to store results
results = []

# Iterate through each colonia
for idx, colonia in colonias.iterrows():
    # Find intersecting isochrones
    intersecting_isochrones = isocrona_30_l[isocrona_30_l.intersects(colonia.geometry)]
    
    # If there are no intersecting isochrones, add 0% coverage and continue
    if intersecting_isochrones.empty:
        results.append({
            'id': colonia['NOMUT'],
            'NOMUT': colonia['NOMUT'],
            'isochrone_coverage': 0
        })
        continue
    
    # Calculate the union of all intersecting isochrones using union_all
    unified_isochrones = intersecting_isochrones.geometry.union_all()
    
    # Calculate the intersection of the unified isochrones with the colonia
    intersection = unified_isochrones.intersection(colonia.geometry)
    
    # Calculate the percentage of coverage
    coverage_percentage = (intersection.area / colonia.geometry.area) * 100
    
    # Ensure the coverage doesn't exceed 100%
    coverage_percentage = min(coverage_percentage, 100)
    
    # Append the result
    results.append({
        'id': colonia['NOMUT'],
        'NOMUT': colonia['NOMUT'],
        'isochrone_coverage': coverage_percentage
    })

# Create a DataFrame from the results
result_df_30 = pd.DataFrame(results)


In [99]:
# Initialize a list to store results
results = []

# Iterate through each colonia
for idx, colonia in colonias.iterrows():
    # Find intersecting isochrones
    intersecting_isochrones = isocrona_45_l[isocrona_45_l.intersects(colonia.geometry)]
    
    # If there are no intersecting isochrones, add 0% coverage and continue
    if intersecting_isochrones.empty:
        results.append({
            'id': colonia['NOMUT'],
            'NOMUT': colonia['NOMUT'],
            'isochrone_coverage': 0
        })
        continue
    
    # Calculate the union of all intersecting isochrones using union_all
    unified_isochrones = intersecting_isochrones.geometry.union_all()
    
    # Calculate the intersection of the unified isochrones with the colonia
    intersection = unified_isochrones.intersection(colonia.geometry)
    
    # Calculate the percentage of coverage
    coverage_percentage = (intersection.area / colonia.geometry.area) * 100
    
    # Ensure the coverage doesn't exceed 100%
    coverage_percentage = min(coverage_percentage, 100)
    
    # Append the result
    results.append({
        'id': colonia['NOMUT'],
        'NOMUT': colonia['NOMUT'],
        'isochrone_coverage': coverage_percentage
    })

# Create a DataFrame from the results
result_df_45 = pd.DataFrame(results)


In [100]:
joined_df = pd.merge(result_df, colonias, on='NOMUT')
joined_df_30 = pd.merge(result_df_30, colonias, on='NOMUT')
joined_df_45 = pd.merge(result_df_45, colonias, on='NOMUT')


joined_df = gpd.GeoDataFrame(joined_df, geometry='geometry')
joined_df_30 = gpd.GeoDataFrame(joined_df_30, geometry='geometry')
joined_df_45 = gpd.GeoDataFrame(joined_df_45, geometry='geometry')


In [101]:
sintesis = sintesis[['colonia', 'C_US']]
sintesis = sintesis.rename(columns={"colonia": "NOMUT"})



In [102]:
joined_df = pd.merge(joined_df, sintesis, on='NOMUT')
joined_df_30 = pd.merge(joined_df_30, sintesis, on='NOMUT')
joined_df_45 = pd.merge(joined_df_45, sintesis, on='NOMUT')

densisdad = densisdad.rename(columns={"colonia": "NOMUT"})
new_df = densisdad[['NOMUT', 'DenPob20']]

joined_df = joined_df.merge(new_df, on = "NOMUT")
joined_df_30 = joined_df_30.merge(new_df, on="NOMUT")
joined_df_45 = joined_df_45.merge(new_df, on = "NOMUT")

In [103]:
# Apply log transformation to density values
joined_df["log_DenPob20"] = np.log(joined_df["DenPob20"] + 1)
joined_df_30["log_DenPob20"] = np.log(joined_df_30["DenPob20"] + 1)
joined_df_45["log_DenPob20"] = np.log(joined_df_45["DenPob20"] + 1)

# Calculate per_pob based on log-transformed density
joined_df["per_pob"] = joined_df["isochrone_coverage"] * joined_df["log_DenPob20"]
joined_df_30["per_pob"] = joined_df_30["isochrone_coverage"] * joined_df_30["log_DenPob20"]
joined_df_45["per_pob"] = joined_df_45["isochrone_coverage"] * joined_df_45["log_DenPob20"]

# Normalize the per_pob column
joined_df['per_pob_n'] = (joined_df["per_pob"] - joined_df["per_pob"].min()) / (joined_df["per_pob"].max() - joined_df["per_pob"].min())
joined_df['rezago_n'] = (joined_df["C_US"] - joined_df['C_US'].min()) / (joined_df['C_US'].max() - joined_df['C_US'].min())

joined_df_30['per_pob_n'] = (joined_df_30["per_pob"] - joined_df_30["per_pob"].min()) / (joined_df_30["per_pob"].max() - joined_df_30["per_pob"].min())
joined_df_30['rezago_n'] = (joined_df_30["C_US"] - joined_df_30['C_US'].min()) / (joined_df_30['C_US'].max() - joined_df_30['C_US'].min())

joined_df_45['per_pob_n'] = (joined_df_45["per_pob"] - joined_df_45["per_pob"].min()) / (joined_df_45["per_pob"].max() - joined_df_45["per_pob"].min())
joined_df_45['rezago_n'] = (joined_df_45["C_US"] - joined_df_45['C_US'].min()) / (joined_df_45['C_US'].max() - joined_df_45['C_US'].min())

In [104]:
joined_df.to_file("joined_df.shp")
joined_df_30.to_file("joined_df_30.shp")
joined_df_45.to_file("joined_df_45.shp")